##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications

# 1. Load CIFAR-10 
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Labels and types 
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# 2.  Data Augmentation 
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1), 
], name="augmentation")

# 3. Load MobileNetV2 with HIGHER Resolution
mobilenet_base = applications.MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3) 
)
mobilenet_base.trainable = False  # Initial frozen state for feature extraction

# 4. Build Model (Integrating Resizing and Preprocessing)
model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(128, 128, interpolation="bilinear"), 
    layers.Lambda(applications.mobilenet_v2.preprocess_input), 
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'), 
    layers.Dropout(0.4), 
    layers.Dense(10)
], name="cifar10_mobilenetv2_updated")

# 5. Phase 1: Feature Extraction
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Using more epochs for the initial head training
history = model.fit(x_train, y_train, epochs=10, validation_split=0.1, batch_size=64)

# 6. Phase 2: DEEPER Fine-Tuning
mobilenet_base.trainable = True
for layer in mobilenet_base.layers[:-50]:
    layer.trainable = False


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Fine-tuning
history_ft = model.fit(x_train, y_train, epochs=10, validation_split=0.1, batch_size=64)

Model: "cifar10_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Starting Feature Extraction (Frozen Base)...
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - accuracy: 0.6074 - loss: 1.1581 - val_accuracy: 0.8098 - val_loss: 0.5506
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.7550 - loss: 0.7098 - val_accuracy: 0.8418 - val_loss: 0.4570
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.7657 - loss: 0.6789 - val_accuracy: 0.8502 - val_loss: 0.4317
Trainable layers in backbone: 20 / 154

Starting Fine-Tuning...
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 36s 41ms/step - accuracy: 0.6446 - loss: 1.1141 - val_accuracy: 0.8240 - val_loss: 0.5553
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.7458 - loss: 0.7640 - val_accuracy: 0.8432 - val_loss: 0.4887
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.7673 - loss: 0.6874 - val_accuracy: 0.8502 - val_loss: 0.4708


In [3]:
# Evaluation
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nUpdated MobileNetV2 Test Accuracy: {test_acc:.4f}")


MobileNetV2 (fine-tuned) test accuracy: 0.8381999731063843
